In [4]:
import pickle
import numpy as np
import pandas as pd
import gc
import sys
from surprise import Dataset
from sklearn.preprocessing import MinMaxScaler
from surprise import SVD
from surprise import Reader
from surprise.model_selection import train_test_split 

In [5]:
df_steam_games = pd.read_csv('../datasets/df_steam_games_clean.csv')
df_user_reviews = pd.read_csv('../datasets/df_user_reviews_clean.csv')
df_user_items = pd.read_csv('../datasets/df_user_items_clean.csv.gz',compression='gzip')

In [6]:
df_user_reviews.to_csv("user_reviews_ur.csv", index=False)

In [7]:
merged_df = df_steam_games.merge(df_user_reviews, left_on='id', right_on='item_id', how='inner')

In [14]:
df_user_reviews = merged_df

In [15]:
df_user_reviews.head()

,genres,title,release_date,price,id,developer,user_id,item_id,recommend,sentiment_analysis,rating
0,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,1997,9.99,282010.0,Stainless Games Ltd,InstigatorAU,282010,1.0,0.5,4.0
1,['Action'],Half-Life,1998,9.99,70.0,Valve,EizanAratoFujimaki,70,1.0,1.0,5.0
2,['Action'],Half-Life,1998,9.99,70.0,Valve,GamerFag,70,1.0,0.0,3.0
3,['Action'],Half-Life,1998,9.99,70.0,Valve,76561198020928326,70,1.0,1.0,5.0
4,['Action'],Half-Life,1998,9.99,70.0,Valve,Bluegills,70,1.0,1.0,5.0


In [33]:
para_entrenar = merged_df[['user_id','title','rating']]

In [35]:
reader = Reader(rating_scale=(0, 4))

data = Dataset.load_from_df(para_entrenar, reader)

In [36]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

## creo la columna rating

In [12]:


# creo la columna rating

# Paso 1: Convertir los valores booleanos a enteros
df_user_reviews['recommend'] = df_user_reviews['recommend'].replace({True: 1, False: 0})

# Paso 2: Normalizar las columnas de recomendación y análisis de sentimientos
scaler = MinMaxScaler()
scaler.fit(df_user_reviews[['recommend', 'sentiment_analysis']])
df_user_reviews[['recommend', 'sentiment_analysis']] = scaler.transform(df_user_reviews[['recommend', 'sentiment_analysis']])

# Paso 3: Crear la columna de calificación
df_user_reviews['rating'] = df_user_reviews['recommend'] * 0.6 + df_user_reviews['sentiment_analysis'] * 0.4

# Paso 4: Ajustar la columna de calificación
df_user_reviews['rating'] = df_user_reviews['rating'] * 5

# Verificar el resultado
df_user_reviews["rating"].unique()


array([4., 5., 3., 2., 0., 1.])

In [37]:
df_user_reviews

,genres,title,release_date,price,id,developer,user_id,item_id,recommend,sentiment_analysis,rating
0,"['Action', 'Indie', 'Racing']",Carmageddon Max Pack,1997,9.99,282010.0,Stainless Games Ltd,InstigatorAU,282010,1.0,0.5,4.0
1,['Action'],Half-Life,1998,9.99,70.0,Valve,EizanAratoFujimaki,70,1.0,1.0,5.0
2,['Action'],Half-Life,1998,9.99,70.0,Valve,GamerFag,70,1.0,0.0,3.0
3,['Action'],Half-Life,1998,9.99,70.0,Valve,76561198020928326,70,1.0,1.0,5.0
4,['Action'],Half-Life,1998,9.99,70.0,Valve,Bluegills,70,1.0,1.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...
49468,['Action'],Counter-Strike: Condition Zero,2004,9.99,80.0,Valve,76561198023508728,80,0.0,0.5,1.0
49469,['Action'],Counter-Strike: Condition Zero,2004,9.99,80.0,Valve,Lone_walker,80,1.0,1.0,5.0
49470,['Action'],Counter-Strike: Condition Zero,2004,9.99,80.0,Valve,green290,80,1.0,0.5,4.0
49471,['Action'],Counter-Strike: Condition Zero,2004,9.99,80.0,Valve,174gamecuman700kngkakak,80,1.0,0.5,4.0


In [38]:
df_user_reviews["title"]

0                  Carmageddon Max Pack
1                             Half-Life
2                             Half-Life
3                             Half-Life
4                             Half-Life
                      ...              
49468    Counter-Strike: Condition Zero
49469    Counter-Strike: Condition Zero
49470    Counter-Strike: Condition Zero
49471    Counter-Strike: Condition Zero
49472    Counter-Strike: Condition Zero
Name: title, Length: 49473, dtype: object

In [ ]:
# nuevo_df = df_user_reviews[['user_id','title','rating']]

In [ ]:
# reader = Reader(rating_scale=(0, 4))

# data = Dataset.load_from_df(nuevo_df, reader)

In [22]:

from surprise import SVD
model = SVD()

In [23]:
# Entrenamos el modelo
model.fit(train_data)

In [24]:
# Predecimos
predictions = model.test(test_data)

In [27]:
predictions[1]

Prediction(uid='76561198074783932', iid='Rocket League®', r_ui=5.0, est=4, details={'was_impossible': False})

In [18]:
# Hacemos una predicción al azar para usuario y juego
model.predict("76561198081529182","Killing Floor")

Prediction(uid='76561198081529182', iid='Killing Floor', r_ui=None, est=4, details={'was_impossible': False})

In [20]:
def recomendacion_usuario(user_id):
  # Obtener todos los juegos que el usuario ha calificado
  juegos_usuario = df_user_reviews[df_user_reviews['user_id'] == user_id]['item_id'].values
  
  # Obtener todos los juegos posibles
  todos_los_juegos = df_user_reviews['item_id'].unique()
  
  # Obtener las predicciones para cada juego que el usuario no ha calificado
  predicciones = [model.predict(user_id, juego) for juego in todos_los_juegos if juego not in juegos_usuario]
  
  # Ordenar las predicciones por puntuación de predicción
  predicciones_ordenadas = sorted(predicciones, key=lambda x: x.est, reverse=True)
  
  # Devolver los 5 juegos recomendados
  return [pred.iid for pred in predicciones_ordenadas[:5]]


In [21]:
recommend_function = recomendacion_usuario("76561198081529182")

In [22]:
recommend_function

[282010, 70, 2400, 3800, 1520]

In [39]:
def recomendacion_usuario(user_id: str):
    try:
        df_user_reviews = pd.read_csv(
            'user_reviews_ur.csv')
    except FileNotFoundError:
        return {"error": "Data files not found"}
    # Obtener todos los juegos que el usuario ha calificado
    juegos_usuario = df_user_reviews[df_user_reviews['user_id']
                                        == user_id]['item_id'].values

    # Obtener todos los juegos posibles
    todos_los_juegos = df_user_reviews['item_id'].unique()

    with open('modelo_entrenado.pkl', 'rb') as file:
        model = pickle.load(file)

    # Obtener las predicciones para cada juego que el usuario no ha calificado
    predicciones = [model.predict(
        user_id, juego) for juego in todos_los_juegos if juego not in juegos_usuario]

    # Ordenar las predicciones por puntuación de predicción
    predicciones_ordenadas = sorted(
        predicciones, key=lambda x: x.est, reverse=True)

    # Devolver los 5 juegos recomendados
    respuesta = []
    diccionario = {}
    for pred in predicciones_ordenadas[:5]:
        respuesta.append(pred.iid)

    for i, juego in enumerate(respuesta, start=1):
        opcion = f"opcion{i}"
        nombre_juego = df_user_reviews.loc[df_user_reviews['item_id'] == juego, 'title'].values[0]
        #nombre_juego = todos_los_juegos.loc[todos_los_juegos["item_id"] == juego, "title"].values[0]
        diccionario[opcion] = {"id": juego, "nombre": nombre_juego}

    return diccionario

In [ ]:
recomendacion_usuario("Sp3ctre")

In [ ]:
# for recomendacion in recomendaciones:
#         listado.append(recomendacion.iid)

#     #Creamos un diccionario con clave-valor para retornar los juegos recomendados
#     for i, juego in enumerate(listado, start=1):
#         opcion = f"opcion{i}"
#         diccionario[opcion] = juego

In [49]:
def recomendacion_usuario(user_id: str):
    try:
        df_user_reviews = pd.read_csv(
            'user_reviews_ur.csv')
    except FileNotFoundError:
        return {"error": "Data files not found"}
    # Obtener todos los juegos que el usuario ha calificado
    juegos_usuario = df_user_reviews[df_user_reviews['user_id']
                                     == user_id]['item_id'].values

    # Obtener todos los juegos posibles
    todos_los_juegos = df_user_reviews['item_id'].unique()

    with open('modelo_entrenado.pkl', 'rb') as file:
        model = pickle.load(file)

    # Obtener las predicciones para cada juego que el usuario no ha calificado
    predicciones = [model.predict(
        user_id, juego) for juego in todos_los_juegos if juego not in juegos_usuario]

    # Ordenar las predicciones por puntuación de predicción
    predicciones_ordenadas = sorted(
        predicciones, key=lambda x: x.est, reverse=True)

    respuesta = []
    diccionario = {}
    for pred in predicciones_ordenadas[:5]:
        respuesta.append(pred.iid)

    for i, juego in enumerate(respuesta, start=1):
        opcion = f"opcion {i}"
        diccionario[opcion] = str(juego)

    return diccionario

In [50]:
recomendacion_usuario("Sp3ctre")

{'opcion 1': '1250',
 'opcion 2': '22200',
 'opcion 3': '43110',
 'opcion 4': '251610',
 'opcion 5': '227300'}

In [47]:
def recomendacion_usuario(user_id: str):
    try:
        df_user_reviews = pd.read_csv('user_reviews_ur.csv')
    except FileNotFoundError:
        return {"error": "Data files not found"}

    # Obtener todos los juegos que el usuario ha calificado
    juegos_usuario = df_user_reviews[df_user_reviews['user_id'] == user_id]['item_id'].values

    # Obtener todos los juegos posibles
    todos_los_juegos = df_user_reviews['item_id'].unique()

    with open('modelo_entrenado.pkl', 'rb') as file:
        model = pickle.load(file)

    # Obtener las predicciones para cada juego que el usuario no ha calificado
    predicciones = [model.predict(user_id, juego) for juego in todos_los_juegos if juego not in juegos_usuario]

    # Ordenar las predicciones por puntuación de predicción
    predicciones_ordenadas = sorted(predicciones, key=lambda x: x.est, reverse=True)

    respuesta = []
    diccionario = {}
    for pred in predicciones_ordenadas[:5]:
        respuesta.append(pred.iid)

    for i, juego in enumerate(respuesta, start=1):
        opcion = f"opcion {i}"
        nombre_juego = df_user_reviews.loc[df_user_reviews['item_id'] == juego, 'title'].values[0]
        diccionario[opcion] = nombre_juego

    return diccionario


In [48]:
recomendacion_usuario("Sp3ctre")

KeyError: 'title'